In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
from scipy.stats import gamma, uniform


In [226]:
disasters = genfromtxt('disasters.csv', delimiter = ',')

def init_mcmc(N, num_breakpoints, psi, rho):

    
    #Prior for t is a logic statement
    # If t is sorted the prior is proportional to the sumproduct of differences
    # returns zero if not sorted
    prior_t_log = lambda t: np.sum(np.log(np.diff(t))) if np.all(t[:-1] <= t[1:]) else 0
    
    #The probability of tau given parameters
    f_tau_log = lambda t, lambdas, nt: -np.sum(np.diff(t)*lambdas) + np.sum(nt*np.log(lambdas))
    
    #Initialize arrays
    breakpoints = np.arange(1658,1981, (1980-1658)/(num_breakpoints+1))
    tau = disasters
    
    burn_in = 0
    M = N + burn_in

    lambdas = np.zeros((M, num_breakpoints + 1))
    thetas = np.zeros((M, num_breakpoints + 1))
    
    prob_breakpoints = np.zeros((M, num_breakpoints))
    

    for index in range(M):
        print(f'Iter: {index}')
        thetas[index] = gamma(2, psi).rvs(num_breakpoints + 1)
        for section in range(num_breakpoints + 1):
            lambdas[index, section] = gamma(2, thetas[index, section]).rvs(1)

        print(f' Theta: {thetas[index]}')
        print(f' Lambda: {lambdas[index]}')
        #Random walk proposal
        for t in np.arange(num_breakpoints) + 1:
            print(f'Breakpoint {t}')
            
            ti = breakpoints[t]
            R = rho*(breakpoints[t+1] - breakpoints[t-1])
            epsilon = np.random.uniform(-R, R)
            ti_proposal = ti+epsilon
            breakpoints_propose = np.copy(breakpoints)
            breakpoints_propose[t] = ti_proposal
            
            print(f' Current breakpoints: {breakpoints}')
            print(f' Proposed breakpoints: {breakpoints_propose}')
            
            if(np.all(breakpoints_propose[:-1] <= breakpoints_propose[1:])):
            
                #Num incidents per interval
                nt_current = np.histogram(tau, bins = breakpoints)[0].astype('int32')
                print(f'nt current: {nt_current}')
                nt_propose = np.histogram(tau, bins = breakpoints_propose)[0].astype('int32')
                print(f'nt propose: {nt_propose}')
                
                
                #Probability of accept
                p_propose_log = prior_t_log(breakpoints_propose) + f_tau_log(breakpoints_propose, lambdas, nt_propose)
                print(f'p propose: {np.exp(p_propose_log)}')
                p_current_log = prior_t_log(breakpoints) + f_tau_log(breakpoints, lambdas, nt_current)
                print(f'p current: {np.exp(p_current_log)}')
                p_accept = np.minimum(1, np.exp(p_propose_log-p_current_log))

                accept = uniform(0,1).rvs() < p_accept
                print(f' P(accept) = {p_accept}')

                if accept:
                    breakpoints = breakpoints_propose

    return breakpoints
    

In [240]:
init_mcmc(2, 1, 0.01, 0.5)

1.1179667077833884e-110

In [190]:
init_mcmc(1000, 1, 1, 0.5)

Iter: 0
 Theta: [3.71094296 2.222361  ]
 Lambda: [6.38640503 4.67589004]
Breakpoint 1
 Current breakpoints: [1658.        1745.4437508 1980.       ]
 Proposed breakpoints: [1658.        1745.4437508 1980.       ]
Iter: 1
 Theta: [4.4585214  3.33866827]
 Lambda: [9.58416302 4.94239233]
Breakpoint 1
 Current breakpoints: [1658.         1641.24486885 1980.        ]
 Proposed breakpoints: [1658.         1641.24486885 1980.        ]
Iter: 2
 Theta: [5.23694373 3.42124092]
 Lambda: [8.35341573 3.97885792]
Breakpoint 1
 Current breakpoints: [1658.         1663.85878793 1980.        ]
 Proposed breakpoints: [1658.         1663.85878793 1980.        ]
Iter: 3
 Theta: [5.02263064 2.87240562]
 Lambda: [7.51439156 3.86170168]
Breakpoint 1
 Current breakpoints: [1658.         1510.53521739 1980.        ]
 Proposed breakpoints: [1658.         1510.53521739 1980.        ]
Iter: 4
 Theta: [1.56203048 4.21632903]
 Lambda: [3.85129044 6.69371512]
Breakpoint 1
 Current breakpoints: [1658.         1467.00

<ipython-input-188-4a6bbc5ef337>:14: RuntimeWarning: overflow encountered in power
  f_tau = lambda t, lambdas, nt: np.exp(-np.sum(np.diff(t)*lambdas))*np.prod(lambdas**nt)



 Lambda: [2.60912409 2.84017329]
Breakpoint 1
 Current breakpoints: [1658.         1225.34699124 1980.        ]
 Proposed breakpoints: [1658.         1225.34699124 1980.        ]
Iter: 67
 Theta: [5.16844762 2.34957045]
 Lambda: [6.90838089 3.34220524]
Breakpoint 1
 Current breakpoints: [1658.         1303.24593473 1980.        ]
 Proposed breakpoints: [1658.         1303.24593473 1980.        ]
Iter: 68
 Theta: [2.68186154 2.83998858]
 Lambda: [5.03979399 3.56346944]
Breakpoint 1
 Current breakpoints: [1658.         1142.46635603 1980.        ]
 Proposed breakpoints: [1658.         1142.46635603 1980.        ]
Iter: 69
 Theta: [1.75137137 2.17343119]
 Lambda: [4.00210834 3.27110886]
Breakpoint 1
 Current breakpoints: [1658.         1126.22839015 1980.        ]
 Proposed breakpoints: [1658.         1126.22839015 1980.        ]
Iter: 70
 Theta: [4.42248964 1.85778768]
 Lambda: [5.18944473 2.614279  ]
Breakpoint 1
 Current breakpoints: [1658.        1177.6087254 1980.       ]
 Proposed 

array([1658.        , 4023.94159713, 1980.        ])

In [113]:
num_breakpoints = 1
breakpoints = np.arange(1658,1981, (1980-1658)/(num_breakpoints+1))
nt = np.histogram(disasters, bins = breakpoints)[0].astype('int32')
lambdas = np.array([1, 2])

array([ 79, 672])

In [163]:
np.arange(1658,1981, (1980-1658)/(num_breakpoints+1))

array([1658., 1819., 1980.])

In [174]:
np.arange(1)

array([0])